## Classifying Audio Vs Visual labels 

In [96]:
%run ../preprocessing/Artifact_Removal/preprocecssing_helpers.ipynb
%run ../preprocessing/StimCodes.ipynb
%run ../Classification/ConcatEpochTrails.ipynb
import mne 

from mne.decoding import Vectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.svm import SVC  # noqa
from sklearn.model_selection import ShuffleSplit  # noqa

from sklearn.metrics import confusion_matrix
from mne.viz import tight_layout

In [68]:
path = 'E:\EpochedEEG'
epoch_files = os.listdir(path)
file = os.path.join(path, epoch_files[0])

In [69]:
# read epoch object 
epoch = mne.read_epochs(file, preload= True)

Reading E:\EpochedEEG\20131216_1441_epo.fif ...
    Found the data of interest:
        t =    -199.22 ...     500.00 ms
        0 CTF compensation matrices available
5424 matching events found
Applying baseline correction (mode: mean)
5424 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated


In [75]:
# get indicies of trials of epoch object
trial_index_list = get_trial_index_list(epoch)

In [97]:
# get new events for epoch object
new_event_list = convert_event_ids_to_stim_combinations(epoch_object=epoch,
                                                        trial_index_list = trial_index_list,
                                                        stim_combinations = stim_combinations)

events shape: (5424, 3)

Could not find stim-code combination.
error in trial index: (0, 1)
Could not find stim-code combination.
error in trial index: (289, 297)
(640, 1)


In [46]:
# assign new events to current epoch object
epoch.events = new_event_list

# assign new event_ids to current epoch object (dictionary 
# found in ../Classification/ConcatEpochTrails.ipynb)
 
epoch.event_id = modality_lexicality_event_ids

In [48]:
# classification time!

# Isolate audio vs visual codes
# audio codes are < 700, # visual codes are > 700 
# audio : 100 , visual : 101
for event in epoch.events:
    if event[-1] < 700:
        event[-1] = 100
    else:
        event[-1] = 101

In [51]:
# classification pipeline
clf = make_pipeline(Vectorizer(),
                    MinMaxScaler(),
                    LogisticRegression(penalty='l1'))

cv = StratifiedKFold(n_splits=10, shuffle = True, random_state = 42)

labels = epoch.events[:,-1]
preds = np.empty(len(epoch._data))
for train, test in cv.split(epoch._data, labels):
    clf.fit(epoch._data[train], labels[train])
    preds[test] = clf.predict(epoch.get_data()[test])

target_names = ['Audio', 'Visual']
report = classification_report(labels, preds, target_names=target_names)
print(report)

ValueError: Found input variables with inconsistent numbers of samples: [5424, 5407]

In [53]:
epoch._data.shape

(5424, 129, 180)

In [54]:
epoch.events.shape

(5407, 3)

In [64]:
new_event_list.shape

(5407, 3)

In [72]:
epoch.events.shape

(5424, 3)

In [98]:
new_event_list.shape

(640,)

In [77]:
debug_epoch = epoch[289:297]

In [79]:
debug_epoch.events

array([[23719,     0,    31],
       [23721,     0,   145],
       [23722,   145,   158],
       [23728,     0,    14],
       [23857,     0,    21],
       [23982,     0,     4],
       [23996,     0,   131],
       [24116,     0,   134]])

array([[  2411,      0,    131],
       [  2580,      0,     31],
       [  2581,     31,    144],
       ...,
       [658880,      0,      1],
       [658931,      0,    131],
       [659013,      0,    134]])

In [84]:
np.array(trial_index_list).shape

(640, 2)

array([array([[2411,    0,  131]]),
       array([[2580,    0, 1012],
       [2581, 1012, 1012],
       [2583,    0, 1012],
       [2593,    0, 1012],
       [2718,    0, 1012],
       [2847,    0, 1012],
       [3200,    0, 1012],
       [3201, 1012, 1012],
       [3203,    0, 1012]]),
       array([[3669,    0, 1012],
       [3671,    0, 1012],
       [3672, 1012, 1012],
       [3682,    0, 1012],
       [3807,    0, 1012],
       [3936,    0, 1012],
       [4298,    0, 1012],
       [4299, 1012, 1012],
       [4301,    0, 1012]]),
       array([[4487,    0, 1012],
       [4489,    0, 1012],
       [4491,    0, 1012],
       [4500,    0, 1012],
       [4625,    0, 1012],
       [4754,    0, 1012],
       [4792,    0, 1012],
       [4884,    0, 1012]]),
       array([[5030,    0, 1012],
       [5032,    0, 1012],
       [5033, 1012, 1012],
       [5043,    0, 1012],
       [5168,    0, 1012],
       [5297,    0, 1012],
       [5340,    0, 1012],
       [5426,    0, 1012],
       [5428